# Santander Product Recommendation

### Part 2. Preprocessing

As I could see from EDA, some data has outliers, missing values and wrong data type. They should be cleaned and split for cross validation to train the model.

### 2-1.  Assemble data together

For preprocessing all data, we assemble train data and test data.<br>
Also, I removed data with no product data, which will not be helpful for training the model.  As test data is provided for prediction, it doesn't have product data. For now, I filled the data as 0 for preprocessing.

In [1]:
from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np
import pickle

trn = pd.read_csv('input/train_ver2.csv')
tst = pd.read_csv('input/test_ver2.csv')

C:\Users\thisi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\thisi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
prods = trn.columns[24:]

trn[prods] = trn[prods].fillna(0.0).astype(np.int8)
no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

for col in trn.columns[24:]:
    tst[col] = 0
    
df = pd.concat([trn, tst], axis=0)
del trn, tst

df.shape

(12020685, 48)

### 2-2.  Cleaning Data

In this part, I removed unnecessary/duplicated data for better prediction.<br>

#### Drop columns
From EDA, I knew 'nomprov' is duplicated and 'tipodom' has 0 standard derivation.<br>
Also, over 90% of values are missing in the features, I removed the data as well.

In [3]:
drop_ = ['nomprov', 'tipodom']

mask = (df.iloc[:,:24].isnull().sum() / df.shape[0]) > 0.9
drop_ += mask[mask].index.tolist()

df.drop(drop_, axis=1, inplace=True)
df.shape

(12020685, 44)

#### Categorical columns

First, I corrected data types, remove noise data, and fill missing values as -99 for penalty. <br>
Then I labeled these columns to convert string to numeric value.

In [4]:
features = []

# Converting data type
flt_to_obj = ['ind_nuevo', 'indrel', 'ind_actividad_cliente']
df[flt_to_obj] = df[flt_to_obj].astype('O')

# Remove noise
df['indrel_1mes'].replace({'1.0':1, '1':1, 2.0:2, '2.0':2,'2':2, 3.0:3, '3':3, '3.0': 3, '4':4, 4.0:4,'4.0': 4, 'P':5}, inplace=True)
df['tiprel_1mes'].replace('N', 'I', inplace=True)

# Filling missing values and factorize it
categorical_cols = ['ind_empleado', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'indfall', 'segmento', 'indrel_1mes'] + flt_to_obj + ['pais_residencia', 'canal_entrada','cod_prov']
df[categorical_cols] = df[categorical_cols].fillna(-99)

for col in categorical_cols:
    df[col], _ = df[col].factorize()
    
features += categorical_cols

#### Numerical columns

For numerical columns, I changed the data type to int and fill missing value and replace outliers as -99 for penalty.

In [5]:
#데이터 타입 숫자로 수정
def str_to_int(series):    
    if type(series) is str:
        return int(series.strip())
    else:
        return series

#수치형 변수 1:age
df['age'].replace(' NA',-99, inplace=True)
df['age'] = df['age'].apply(str_to_int).astype(np.int8)

#수치형 변수 2:antiguedad
df['antiguedad'].replace(['     NA','-999999',-999999], -99, inplace=True)
df['antiguedad'] = df['antiguedad'].apply(str_to_int).astype(np.int8)

#수치형 변수3: renta
df['renta'].fillna(-99, inplace=True)
df['renta'].replace('         NA', -99, inplace=True)
df['renta'] = df['renta'].apply(float).round()

# 학습에 사용할 수치형 변수를 features에 추구한다.
features += ['age', 'antiguedad' ,'renta']

<br>

### 2-3.  Feature Engineering

Created lagged-features(1 month behind) and added them on the feature list.

In [6]:
# 날짜를 숫자로 변환하는 함수이다. 2015-01-28은 1, 2016-06-28은 18로 변환된다
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] 
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

# 날짜를 숫자로 변환하여 int_date에 저장한다
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

# 데이터를 복사하고, int_date 날짜에 1을 더하여 lag를 생성한다. 변수명에 _prev를 추가한다.
df_lag = df.copy()
df_lag.columns = [col+'_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns]
df_lag['int_date'] += 1

# 원본 데이터와 lag 데이터를 ncodper와 int_date 기준으로 합친다. Lag 데이터의 int_date는 1 밀려 있기 때문에, 저번 달의 제품 정보가 삽입된다.
df_trn = df.merge(df_lag, on=['ncodpers','int_date'], how='left')

# 메모리 효율을 위해 불필요한 변수를 메모리에서 제거한다
del df, df_lag

# 저번 달의 제품 정보가 존재하지 않을 경우를 대비하여 0으로 대체한다.
df_trn.fillna(0, inplace=True)

# lag-1 변수를 추가한다.
features += [f+'_prev' for f in features]
features += [p+'_prev' for p in prods]

<br>
Additional features will be month and year from 'fecha_alta', which indicates the date customer opened account for first time.

In [7]:
# 날짜 변수에서 연도와 월 정보를 추출한다.
df_trn['fecha_alta_month'] = df_trn['fecha_alta'].map(lambda x: 0.0 if x.__class__ in [int,float] else float(x.split('-')[1])).astype(np.int8)
df_trn['fecha_alta_year'] = df_trn['fecha_alta'].map(lambda x: 0.0 if x.__class__ in [int,float] else float(x.split('-')[0])).astype(np.int16)

features += ['fecha_alta_month', 'fecha_alta_year']

<br>

### 2-4. Data Split

After cleaning data, I split data into train/test for cross validation.<br>

In [8]:
vld_date = '2016-05-28'
tst_date = '2016-06-28'

tst_all = df_trn[df_trn['fecha_dato'] == tst_date]
trn = df_trn[df_trn['fecha_dato']< tst_date]

del df_trn

In [9]:
with open('input/vlds.pkl', 'wb') as f:
    pickle.dump({
        'vld_all': trn
    }, f)

For validation process, I extracted the data of customers who actually buy new products. <br>
Based on this, I chose 16 products newly purchased than other products, which will be target class.<br>
Other 8 classes and non-additional data will be 17th class.

In [10]:
# 훈련 데이터에서 신규 구매 건수만 추출한다.
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i
    X.append(prX)
    Y.append(prY)

XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

In [11]:
target = sorted(XY[XY['fecha_dato']>'2015-05-28']['y'].value_counts().index.tolist()[:16])
non_target = sorted(XY[XY['fecha_dato']>'2015-05-28']['y'].value_counts().index.tolist()[16:])
target, non_target

([2, 4, 5, 6, 7, 8, 9, 11, 12, 13, 17, 18, 19, 21, 22, 23],
 [0, 1, 3, 10, 14, 15, 16, 20])

In [12]:
prevs = [prod+'_prev' for prod in prods]

arr = trn[prods].values - trn[prevs].values
arr = np.sum(arr, axis=1)
non_index = []
for i in range(len(arr)):
    if arr[i]==0:
        non_index.append(i)
        
trn_non = trn.iloc[non_index]
trn_non['y'] = 24
trn_non['target'] = 16

C:\Users\thisi\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\thisi\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [13]:
tarX=[]
for i,t in enumerate(target):
    tX = XY[XY['y'] == t]
    tX['target'] = i
    tarX.append(tX)

for nt in non_target:
    tX = XY[XY['y'] == nt]
    tX['target'] = 16
    tarX.append(tX)

tarX.append(trn_non)
del trn_non, XY

trn_17 = pd.concat(tarX)

C:\Users\thisi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\thisi\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Lastly, I saved the metadata and processed data as file for use in future.

In [16]:
with open('input/meta_data.pkl', 'wb') as ff:
    pickle.dump({
        'features':features,
        'prods':prods,
        'target':target
    }, ff)
    
with open('input/processed_data.pkl', 'wb') as fff:
    pickle.dump({
        'trn_all': trn_17,
        'tst_all': tst_all
    }, fff)